In [1]:
# Setup
import os
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from statistics import stdev
from scipy import nanstd
import statsmodels.api as sm
from datetime import date
from dateutil.relativedelta import relativedelta # $ pip install python-dateutil

os.getcwd()

C:\Users\42874\AppData\Local\Continuum\anaconda3\envs\Michal\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\42874\AppData\Local\Continuum\anaconda3\envs\Michal\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
C:\Users\42874\AppData\Local\Continuum\anaconda3\envs\Michal\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


'C:\\Users\\42874\\Documents\\00_Cases\\Paymore'

In [53]:
# Options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)

In [67]:
# Helpers

def relative_date(acq_date, date):
    
    acq_date_m =  pd.to_datetime(acq_date.apply(lambda x: x.strftime('%Y-%m')))
    date_m = pd.to_datetime(date.apply(lambda x: x.strftime('%Y-%m')))
    
    #months_from_acq = (date_m - acq_date_m).astype('timedelta64[M]')
    date_df = pd.DataFrame([acq_date_m, date_m])
    months_from_acq = date_df.apply(lambda x: date_diff_months(x.iloc[0], x.iloc[1]))
    
    output = months_from_acq.apply(lambda x: dt.date(2000,1,1) +  pd.DateOffset(months=x))
    return output


def dates_2_months(x):
    return x.strftime('%Y-%m')


def date_diff_months(start_date, end_date):
    return (end_date.year - start_date.year) * 12 + (end_date.month - start_date.month)


def calc_multiplier_EBIDTA(file):
    
    ebidta_dict = {}
        
    PRACTICES = get_practices(file)

    Practice_db = pd.read_excel(file, sheet_name=PRACTICES, engine='openpyxl', header=None)
    
    for practice in PRACTICES:
        #ebidta_dict[practice] = calc_EBITDA_multiplier(Practice_db[practice])
        ebidta_dict[practice] = get_EBIDTA(Practice_db[practice])
    
    return pd.DataFrame.from_dict(ebidta_dict).transpose()    
    

    
def calc_ExpEBIDTA(file):
    
    ebidta_dict = {}
        
    PRACTICES = get_practices(file)

    Practice_db = pd.read_excel(file, sheet_name=PRACTICES, engine='openpyxl', header=None)
    
    for practice in PRACTICES:
        #ebidta_dict[practice] = calc_EBITDA_multiplier(Practice_db[practice])
        ebidta_dict[practice] = get_ExpEBIDTA(Practice_db[practice])
    
    return pd.DataFrame.from_dict(ebidta_dict).transpose()    



def calc_Revenue(file):
    
    ebidta_dict = {}
        
    PRACTICES = get_practices(file)

    Practice_db = pd.read_excel(file, sheet_name=PRACTICES, engine='openpyxl', header=None)
    
    for practice in PRACTICES:
        #ebidta_dict[practice] = calc_EBITDA_multiplier(Practice_db[practice])
        ebidta_dict[practice] = get_Revenue(Practice_db[practice])
    
    return pd.DataFrame.from_dict(ebidta_dict).transpose()    


def get_practices(file):
    
    sht_names = file.sheet_names
    sht_visibility = [x.sheet_state for x in file._reader.book._sheets]

    sheet_df = pd.DataFrame([sht_names,sht_visibility]).transpose()
    visibility = [x=='visible' for x in sht_visibility]

    sht_names_visible = sheet_df[sheet_df.loc[:,1]=='visible']
    sht_names_visible = sht_names_visible.loc[:,0].to_list()
    PRACTICES = sht_names_visible[12:]
    
    return PRACTICES


def calc_EBIDTA_multiplier(x):
    
    y = x.copy()
    y.loc[8:12,4].replace(0, np.nan, inplace=True)
    
    return [y.loc[4,5] / y.loc[i,4] for i in range(8,13)]


def get_EBIDTA(x):
    
    y = x.copy()
    #y.loc[8:12,4].replace(0, np.nan, inplace=True)
    
    return [y.loc[i,3] for i in range(8,13)]


def get_ExpEBIDTA(x):
    
    y = x.copy()
    #y.loc[8:12,4].replace(0, np.nan, inplace=True)
    
    return [y.loc[i,4] for i in range(8,13)]


def get_Revenue(x):
    
    y = x.copy()
    #y.loc[8:12,4].replace(0, np.nan, inplace=True)
    
    return [y.loc[i,2] for i in range(8,13)]


def change_to_long(df, id_vars, value):
    
    df_copy = df.copy()
    df_long = df_copy.melt(id_vars=id_vars, var_name="Date", value_name=value)
    df_long = df_long[~df_long.loc[:,value].isna()]
    df_long = df_long[~df_long.Acquisition_date.isna()]
    #df_long["Acquisition_date"] = df_long.Acquisition_date.apply(lambda x: x.strftime('%Y-%m-%d'))
    #df_long["Date"] = df_long.Date.apply(lambda x: x.strftime('%Y-%m-%d'))
    df_long["Relative_date"] = relative_date(df_long.Acquisition_date ,df_long.Date)
    
    return df_long


def summarize_tables(df, value):
    
    df_copy = df.copy()
    df_copy["gb_index"] = df_copy.Relative_date.dt.strftime('%Y')
    
    df_copy_gb =  df_copy.groupby(["Name","Bain_ID","gb_index"]).agg({
        value: np.sum
    })
    
    df_copy_gb = df_copy_gb.reset_index()
    
    df_copy_gb["gb_index"] = df_copy_gb["gb_index"].astype(int)
    df_copy_gb["gb_index"] += (-2000)
    
    df_copy_gb = df_copy_gb.rename(columns={"gb_index": "year"})
    
    return df_copy_gb


def populate_modeled_EBITDA(x, EBITDA_mult_table):
    
    multiplier = EBITDA_mult_table[EBITDA_mult_table.Bain_ID==x.Bain_ID]
    
    #return flatten_list([x.total_consideration/multiplier[i] for i in range(0,5)])
    return flatten_list([multiplier[i] for i in range(0,5)])


def flatten_list(t):
    return [item for sublist in t for item in sublist]


def CAGR(df, idx, start, end):
    
    cagr_df = []

    for i in df[idx].unique():
        temp_df = df[df[idx]==i]
        cagr_temp = (temp_df.loc[temp_df.year==end,"EBIDTA"].values/temp_df.loc[temp_df.year==start,"EBIDTA"].values)**(1/(end-start)) -1
        cagr_df.append([i,cagr_temp[0]])

    return pd.DataFrame(cagr_df)



def CAGR_short(start_cell, end_cell, start, end):
       return (end_cell/start_cell)**(1/(end-start))-1


def normalize(x):
    x = x.astype(float)
    return (x-x.mean())/np.nanstd(x)


def run_regression(df, y_col_name, reg_cols):
    Y = df[y_col_name]
    X = df[reg_cols]
    X = sm.add_constant(X)

    model = sm.OLS(Y,X)
    results = model.fit()
    
    return results

In [4]:
# Data import
Master_dataset = pd.read_excel("210921 - Pay more data request - Bain ID AAG v7.xlsx", sheet_name="Master dataset", usecols="C:W", skiprows=3, nrows=162, engine='openpyxl')
BainID_table =  pd.read_excel("210921 - Pay more data request - Bain ID AAG v7.xlsx", sheet_name="Performance post acqn", usecols="A:FA", skiprows=3, nrows=141, engine='openpyxl')

Main_dataset = pd.ExcelFile("July 2021 - Acquisition Summaries - AAG nopass.xlsx", engine='openpyxl')

Revenue_table =  pd.read_excel(Main_dataset, sheet_name="Post Acquisition Data Adj 1stYr", usecols="B:EY", skiprows=3, nrows=141, engine='openpyxl')
GrossProfit_table =  pd.read_excel(Main_dataset, sheet_name="Post Acquisition Data Adj 1stYr", usecols="B:EY", skiprows=154, nrows=141, engine='openpyxl')
EBITDA_table =  pd.read_excel(Main_dataset, sheet_name="Post Acquisition Data Adj 1stYr", usecols="B:EY", skiprows=306, nrows=141, engine='openpyxl')

In [5]:
# Data cleaning
Master_dataset = Master_dataset.iloc[1:,]
Master_dataset = Master_dataset[["Deal date", "Acquisition #", "Practices in deal", "Total consideration","Multiple paid","# chairs / surgeries","Revenue split at acq.","5Y multiple","Sites in deal","Earn out period", "Deferred payment"]].rename(columns={
    "Deal date": "deal_date",
    "Acquisition #": "Bain_ID", 
    "Practices in deal": "Name",
    "Total consideration": "total_consideration",
    "Multiple paid": "multiple_paid",
    "# chairs / surgeries": "No_chairs",
    "Revenue split at acq.": "Revenue_split_at_acq",
    "5Y multiple": "5Y_multiple",
    "Sites in deal": "sites",
    "Earn out period": "earn_out_period", 
    "Deferred payment": "deferred_payment"
})
Master_dataset = Master_dataset[~Master_dataset.total_consideration.isna()]

BainID_table = BainID_table[["Bain_ID","REVENUE"]].rename(columns={
    "REVENUE": "Name"
})

# Tables from Post Acquisition Data Adj 1stYr sheet
Revenue_table = Revenue_table.drop(columns=["Unnamed: 3"])
Revenue_table = Revenue_table.rename(columns={
    "REVENUE": "Name",
    "Unnamed: 2": "Acquisition_date"
})
Revenue_table = Revenue_table.merge(BainID_table, on="Name", how="left")
Revenue_table = Revenue_table.merge(Master_dataset[["Bain_ID","deal_date"]], on="Bain_ID")
Revenue_table[["Bain_ID","Name","Acquisition_date","deal_date"]].to_csv("Acqusition_dates.csv")

Revenue_table["Acquisition_date"] = Revenue_table["deal_date"]
Revenue_table = Revenue_table.drop(columns="deal_date")

GrossProfit_table  = GrossProfit_table .drop(columns=["Unnamed: 2", "Unnamed: 3"])
GrossProfit_table = GrossProfit_table.rename(columns={
    "GROSS PROFIT": "Name"
})
GrossProfit_table = GrossProfit_table.merge(Revenue_table[["Name","Acquisition_date"]], on="Name", how="left")
GrossProfit_table = GrossProfit_table.merge(BainID_table, on="Name", how="left")


EBITDA_table = EBITDA_table.drop(columns=["Unnamed: 2", "Unnamed: 3"])
EBITDA_table = EBITDA_table.rename(columns={
    "EBITDA": "Name"
})
EBITDA_table = EBITDA_table.merge(Revenue_table[["Name","Acquisition_date"]], on="Name", how="left")
EBITDA_table = EBITDA_table.merge(BainID_table, on="Name", how="left")

In [ ]:
#EBITDA multiplier, expected EBIDTA and Historical revenue are read from individual sheets from July 2021 - Acquisition Summaries - AAG nopass.xlsx file.
#Mapping between sheet names and Bain_ID can be found in EBITDA_multiplier_dict.csv

In [6]:
# EBITDA multiplier - it is historical EBIDTA. The name of the variable is the legacy of older analysis.
EBITDA_multiplier = calc_multiplier_EBIDTA(Main_dataset)

# EBITDA_multiplier.to_csv("EBITDA_multiplier_dict.csv") # if new dataset will come
EBITDA_dict = pd.read_csv("EBITDA_multiplier_dict.csv")
EBITDA_dict = EBITDA_dict[~EBITDA_dict.Bain_ID.isna()]

EBITDA_multiplier = EBITDA_multiplier.merge(EBITDA_dict, left_index = True, right_on="Name")

EBITDA_multiplier.to_csv("EBIDTA.csv")

EBITDA_multiplier_long = EBITDA_multiplier.melt(id_vars=["Name","Bain_ID"], var_name="year", value_name="EBITDA_multiplier")


In [7]:
# expected EBIDTA
EBITDA_Exp = calc_ExpEBIDTA(Main_dataset)

# EBITDA_multiplier.to_csv("EBITDA_multiplier_dict.csv") # if new dataset will come
EBITDA_dict = pd.read_csv("EBITDA_multiplier_dict.csv")
EBITDA_dict = EBITDA_dict[~EBITDA_dict.Bain_ID.isna()]

EBITDA_Exp = EBITDA_Exp.merge(EBITDA_dict, left_index = True, right_on="Name")

EBITDA_Exp.to_csv("EBIDTA_EXP.csv")

EBITDA_Exp_long = EBITDA_Exp.melt(id_vars=["Name","Bain_ID"], var_name="year", value_name="EBITDA_multiplier")

In [8]:
# Historical revenue
Revenue_ind = calc_Revenue(Main_dataset)

EBITDA_dict = pd.read_csv("EBITDA_multiplier_dict.csv")
EBITDA_dict = EBITDA_dict[~EBITDA_dict.Bain_ID.isna()]

Revenue_ind  = Revenue_ind .merge(EBITDA_dict, left_index = True, right_on="Name")

Revenue_ind.to_csv("Revenue_ind.csv")

Revenue_ind_long = Revenue_ind .melt(id_vars=["Name","Bain_ID"], var_name="year", value_name="Revenue_ind")

In [9]:
# List of the practices, which were active 3 years before COVID (start oof the pandemic - Mar 1 2020
preCOVID_practices = Revenue_table[Revenue_table.Acquisition_date<"2017-03-01"].Bain_ID.unique()

In [13]:
# long form of tables from Post Acquisition Data Adj 1stYr
Revenue_table_long = change_to_long(Revenue_table, ["Name", "Acquisition_date","Bain_ID"], "Revenue")
GrossProfit_table_long = change_to_long(GrossProfit_table, ["Name", "Acquisition_date","Bain_ID"], "Gross_profit")
EBITDA_table_long = change_to_long(EBITDA_table, ["Name", "Acquisition_date","Bain_ID"], "EBIDTA")

# Note - 2000-01-01 is a starting date. years in 1990s are the data before acquisition, year 2004 e.g. is a 5th year after the acquisition (2000 - 1st, 2001 - 2nd etc.)

In [14]:
# long form of tables from Post Acquisition Data Adj 1stYr - annual summary from monthly data
Revenue_table_summ = summarize_tables(Revenue_table_long,"Revenue")
GrossProfit_table_summ = summarize_tables(GrossProfit_table_long,"Gross_profit")
EBITDA_table_summ = summarize_tables(EBITDA_table_long,"EBIDTA")

In [15]:
summary_table = Revenue_table_summ.merge(GrossProfit_table_summ, on=["Name","year"]).merge(EBITDA_table_summ, on=["Name","year"])

In [17]:

# ????
qwe = Master_dataset.apply(lambda x: populate_modeled_EBITDA(x, EBITDA_multiplier), axis=1)
asd = qwe[qwe.apply(lambda x: len(x))>0]

modeled_EBIDTA=pd.DataFrame()

for i in range(len(asd)):
    modeled_EBIDTA=modeled_EBIDTA.append(pd.DataFrame(asd.iloc[i]).transpose())

modeled_EBIDTA.index = Master_dataset.loc[qwe.apply(lambda x: len(x))>0,:].Bain_ID

modeled_EBIDTA = modeled_EBIDTA.reset_index().melt(id_vars="Bain_ID", var_name="year", value_name="Exp_EBIDTA")

In [65]:
#### EBIDTA analysis file ####
# The file list EBIDTA per year per practice
# Years, which overlap with the pandemic or are after Mar 2020 should be removed.

EBITDA_db = EBITDA_table_summ.drop(columns="EBIDTA").merge(EBITDA_Exp_long.drop(columns="Name"), on=["year","Bain_ID"], how="left").merge(EBITDA_multiplier_long.drop(columns="Name"), on=["year","Bain_ID"], how="left").rename(columns={
    "EBITDA_multiplier_x": "Exp_EBIDTA",
    "EBITDA_multiplier_y": "EBIDTA"
})

EBITDA_db = EBITDA_db[["Name","Bain_ID","year","EBIDTA","Exp_EBIDTA"]]

EBITDA_db = EBITDA_db.sort_values(["Bain_ID","year"])

EBITDA_db["Delta"] = EBITDA_db.Exp_EBIDTA - EBITDA_db.EBIDTA
EBITDA_db["Delta_pct"] = EBITDA_db.Delta / EBITDA_db.Exp_EBIDTA

EBITDA_db = EBITDA_db.merge(Revenue_table[["Bain_ID","Acquisition_date"]], on="Bain_ID", how="left")

EBITDA_db.year += 1
EBITDA_db["Acquisition_date"] = EBITDA_db.apply(lambda x: x.Acquisition_date + relativedelta(years=x.year), axis=1)
EBITDA_db = EBITDA_db[EBITDA_db.Acquisition_date<"2020-03-01"]
EBITDA_db = EBITDA_db[EBITDA_db.year>0]
EBITDA_db = EBITDA_db[EBITDA_db.year<6]
EBITDA_db = EBITDA_db.drop(columns="Acquisition_date")

EBITDA_db.to_csv("EBIDTA_analysis.csv")

In [23]:
#????
EBIDTA_driver = EBITDA_db[EBITDA_db.year==3].merge(EBITDA_db[EBITDA_db.year==1], on=["Bain_ID"])
EBIDTA_driver = EBIDTA_driver[["Bain_ID","EBIDTA_x","EBIDTA_y"]].rename(columns={
    "EBIDTA_x": "EBIDTA_3Y",
    "EBIDTA_y": "EBIDTA_1Y"
})
EBIDTA_driver["diff_EBIDTA"] = EBIDTA_driver["EBIDTA_3Y"] - EBIDTA_driver["EBIDTA_1Y"]

In [25]:
##### ADDITIONAL TABLE #####
# table containing specific data about the practices

ebidta_revenue = EBITDA_multiplier_long.merge(Revenue_ind_long, on=["Bain_ID","year","Name"])
ebidta_revenue_short = ebidta_revenue[ebidta_revenue.year==0].merge(ebidta_revenue[ebidta_revenue.year==2],on=["Name","Bain_ID"])

ebidta_revenue_short = ebidta_revenue_short.drop(columns=["year_x","year_y"]).rename(columns={
    "EBITDA_multiplier_x": "EBIDTA_1Y",
    "EBITDA_multiplier_y": "EBIDTA_3Y",
    "Revenue_ind_x": "Revenue_1Y",
    "Revenue_ind_y": "Revenue_3Y"
})

ebidta_revenue_short.loc[ebidta_revenue_short["EBIDTA_1Y"]==0,"EBIDTA_1Y"]=None
ebidta_revenue_short.loc[ebidta_revenue_short["EBIDTA_3Y"]==0,"EBIDTA_3Y"]=None
ebidta_revenue_short.loc[ebidta_revenue_short["Revenue_1Y"]==0,"Revenue_1Y"]=None
ebidta_revenue_short.loc[ebidta_revenue_short["Revenue_3Y"]==0,"Revenue_3Y"]=None

ebidta_revenue_short["margin_1Y"] = ebidta_revenue_short["EBIDTA_1Y"]/ebidta_revenue_short["Revenue_1Y"]
ebidta_revenue_short["margin_3Y"] = ebidta_revenue_short["EBIDTA_3Y"]/ebidta_revenue_short["Revenue_3Y"]
ebidta_revenue_short["abs_revenue_growth"] = ebidta_revenue_short["Revenue_3Y"] - ebidta_revenue_short["Revenue_1Y"]
ebidta_revenue_short["margin_expansion"] = ebidta_revenue_short["margin_3Y"] - ebidta_revenue_short["margin_1Y"]

ebidta_revenue_short["CAGR_EBIDTA"] = ebidta_revenue_short.apply(lambda x: CAGR_short(x.EBIDTA_1Y, x.EBIDTA_3Y, 1,3), axis=1)
ebidta_revenue_short["diff_EBIDTA"] = ebidta_revenue_short["EBIDTA_3Y"] - ebidta_revenue_short["EBIDTA_1Y"]

ebidta_revenue_short = ebidta_revenue_short.merge(Master_dataset[["Bain_ID", "multiple_paid"]], on ="Bain_ID")
ebidta_revenue_short[ebidta_revenue_short.Bain_ID.isin(preCOVID_practices)].to_csv("additional_table.csv")

In [29]:
#[["Bain_ID","EBIDTA_1Y","Revenue_1Y","margin_1Y","CAGR_EBIDTA","diff_EBIDTA"]]

##### DRIVER database for regressions
driver_db = Master_dataset.merge(ebidta_revenue_short.drop(columns=["Name","multiple_paid"]), on="Bain_ID", how="left")
driver_db = driver_db.drop(columns="deal_date")
driver_db = driver_db.set_index("Bain_ID")
driver_db = driver_db.drop(columns=["Name"])

driver_db = driver_db[driver_db.index.isin(preCOVID_practices)]
driver_db[["multiple_paid","CAGR_EBIDTA","EBIDTA_3Y","diff_EBIDTA"]].to_csv("correlation_db.csv")

driver_db_norm = driver_db.apply(lambda x: normalize(x))
driver_db_norm = driver_db_norm.dropna()

In [71]:
#### REGRESSION #####
y_col_name = ["diff_EBIDTA"]
reg_cols = ["No_chairs", "margin_1Y", "Revenue_1Y", "Revenue_split_at_acq","deferred_payment", "earn_out_period"]

results_reg = run_regression(driver_db_norm, y_col_name, reg_cols)
results_reg.params.to_csv("regression_importance_diff_EBIDTA.csv")

y_col_name = ["CAGR_EBIDTA"]
results_reg = run_regression(driver_db_norm, y_col_name, reg_cols)
results_reg.params.to_csv("regression_importance_CAGR.csv")
